In [1]:
using Pkg
Pkg.activate(@__DIR__);
pkg"instantiate"
pkg"precompile"

using RigidBodyDynamics
using MeshCatMechanisms
using StaticArrays
using RigidBodyDynamics.Contact
using RigidBodySim

  Activating environment at `/mnt/064AC6424AC62E6D/git_workspace/FlyHopper/scripts/Julia/Project.toml`


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-10952388601810113804\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-4781591921898377143\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-10952388601810113804\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

┌ Warning: Error requiring `StaticArrays` from `ArrayInterface`
│   exception = (ErrorException("too many parameters for type"), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x00007f5f41ccf8a9, Ptr{Nothing} @0x00007f5f41d3ae56, Ptr{Nothing} @0x00007f5f41d45769, Ptr{Nothing} @0x00007f5f41d61f15, Ptr{Nothing} @0x00007f5f41d61bce, Ptr{Nothing} @0x00007f5f41d62811, Ptr{Nothing} @0x00007f5f41d63297, Base.InterpreterIP in top-level CodeInfo for ArrayInterface at statement 90, Ptr{Nothing} @0x00007f5f41d7eb31, Ptr{Nothing} @0x00007f5f41d80949, Ptr{Nothing} @0x00007f5ebd906bb1, Ptr{Nothing} @0x00007f5ebd906bdc, Ptr{Nothing} @0x00007f5f41d45769, Ptr{Nothing} @0x00007f5f41d61f15, Ptr{Nothing} @0x00007f5f41d61bce, Ptr{Nothing} @0x00007f5f41d62811, Ptr{Nothing} @0x00007f5f41d62b90, Ptr{Nothing} @0x00007f5f41d6304a, Base.InterpreterIP in MethodInstance for err(::Any, ::Module, ::String) at statement 2, Ptr{Nothing} @0x00007f5ebd906b27, Ptr{Nothing} @0x00007f5ebd906b3c, Ptr{Nothing} @0x0000

In [2]:
curdir = pwd()
const URDFPATH = joinpath(curdir, "../../res/flyhopper_mockup/urdf/flyhopper_mockup_jl.urdf")
bot = parse_urdf(URDFPATH, floating=true)
remove_fixed_tree_joints!(bot);

In [3]:
# add environment
rootframe = root_frame(bot)
ground = HalfSpace3D(Point3D(rootframe, 0., 0., 0.), FreeVector3D(rootframe, 0., 0., 1))
add_environment_primitive!(bot, ground);

In [4]:
g = -9.81 # gravitational acceleration in z-direction
state = MechanismState(bot)
set_configuration!(state, [1; 0; 0; 0; 0; 0; 0.7])
q = configuration(state)
v = velocity(state)

8-element SegmentedVector{JointID, Float64, Base.OneTo{JointID}, Vector{Float64}}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [6]:
build_contact_points(bot)[1]


Point3D in "after_Joint 2": [0, 0, 0]

In [23]:
Nmdl = RigidBodyDynamics.Contact.hunt_crossley_hertz()   
Fmdl = RigidBodyDynamics.Contact.ViscoelasticCoulombModel(0.3, 30e3,0.3)   
scm = RigidBodyDynamics.Contact.SoftContactModel(Nmdl, Fmdl)
point = Point3D(default_frame(findbody(bot, "base_link")), 0, 0, 0)  # build_contact_points(bot)[1]  # 
cp =  RigidBodyDynamics.Contact.ContactPoint(point, scm)

add_contact_point!(findbody(bot, "Link 2"), cp)
contact_points(findbody(bot, "Link 2"))

#=

const DefaultContactPoint{T} = ContactPoint{T,SoftContactModel{HuntCrossleyModel{T},ViscoelasticCoulombModel{T}}}

function add_contact_point!(body::RigidBody{T}, point::DefaultContactPoint{T}) where {T}
    loc = location(point)
    tf = fixed_transform(body, loc.frame, default_frame(body))
    point.location = tf * loc
    push!(body.contact_points, point)
    nothing
end
=#

LoadError: MethodError: no method matching add_contact_point!(::RigidBody{Float64}, ::ContactPoint{Int64, SoftContactModel{HuntCrossleyModel{Float64}, ViscoelasticCoulombModel{Float64}}})
[0mClosest candidates are:
[0m  add_contact_point!(::RigidBody{T}, [91m::DefaultContactPoint{T}[39m) where T at /home/ben/.julia/packages/RigidBodyDynamics/8B04X/src/rigid_body.jl:173

In [9]:
center_of_mass(state)

Point3D in "world": [-0.002547131739826878, -3.3470181064453804e-8, 0.697058817292039]

In [10]:
q0, q1 = joints(bot)
function simple_control!(torques::AbstractVector, t, state::MechanismState)
    # velocity_range: Return the range of indices into the joint velocity vector v corresponding to joint "joint"
    torques[velocity_range(state, q0)] .= -1 .* velocity(state, q0)
    torques[velocity_range(state, q1)] .= -1 .* velocity(state, q1)
end;

In [11]:
final_time = 10.
ts, qs, vs = simulate(state, final_time, simple_control!; Δt = 1e-3);

In [12]:
zero_velocity!(state)
# set_configuration!(state, q0, 0.7)
# set_configuration!(state, q1, -0.8);
mvis = MechanismVisualizer(bot, URDFVisuals(URDFPATH));

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://localhost:8700
└ @ MeshCat /home/ben/.julia/packages/MeshCat/X2AUA/src/visualizer.jl:73


In [13]:
MeshCatMechanisms.animate(mvis, ts, qs; realtimerate = 1.);